In [1]:
from filepicker import *
from eye_tracker_helpers import *
import pandas as pd
import os
from matplotlib import pyplot as plt


def analyseVideo(video, thresh, roi):
    """
    Main analysis function
    :param video: Video class object (video_handling)
    :param thresh: threshold used to find eyes and swimbladder
    :param roi: crop each frame to ROI (if None then video is not cropped)
    :return: pandas DataFrame (frame number and vergence angles)y
    """
    left = []
    right = []

    for frame in range(video.framecount):

        img = video.grabFrameN(frame)
        #img = cv2.medianBlur(img, 9)
        if roi is not None:
            img = cropImage(img, roi)
            
        """
        frameData() returns centres and angles for the fish axis and both eyes.
        To calculate vergence angles:
            - Angles increase CCW and a converged eye points towards the midline
            - A converged left eye has a greater CCW angle than the body axis
            - CW and CCW are reversed
            - Angles greater than 180 degrees mist be divergent
        """
        c, th, l_c, l_th, r_c, r_th = frameData(img, thresh)
        l_verg = findClockwiseAngle(th, l_th)
        r_verg = findClockwiseAngle(r_th, th)
        if l_verg > math.pi:
            l_verg -= 2 * math.pi
        if r_verg > math.pi:
            r_verg -= 2 * math.pi
        left.append(math.degrees(l_verg))
        right.append(math.degrees(r_verg))

    df = pd.DataFrame(dict(left=left, right=right), index=range(video.framecount), columns=['left', 'right'])

    return df


def checkTracking(video, thresh=200, roi=None):
    """
    Check that tracking is working
    :param video: Video class object (video_handling)
    :param thresh: threshold used to find eyes and swimbladder
    :param roi: crop each frame to ROI (if None then video is not cropped)
    :return: None
    """
    winname = video.name
    video.addDisplay(winname, displayFunction=showEyes, displayKwargs={'thresh': thresh, 'roi': roi})
    cv2.waitKey(0)
    video.removeDisplay(winname)


if __name__ == "__main__":
    ### SELECT A FOLDER TO ANALYZE, FIND ALL AVI FILES IN THAT FOLDER ###
    folder = 'D:/001 DATA/28 FEB 2020/2P094/A5/avi/'#pickdir()
    filenames = os.listdir(folder)
    avis = [filename for filename in filenames if os.path.splitext(filename)[1] == '.avi']
    ### CREATE A FOLDER WHERE RESULTS WILL BE SAVED ###
    sample = folder.split('/')[-4] + '-' + folder.split('/')[-3]
    output_folder = 'D:/new folder/002 ANALYSIS/OKN analysis/tuning_curve/' + sample + '/' 
    #output_folder = os.path.join(folder, 'eye-angle')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    ### PROMPTS FOR USING ROI, CHANGING DEFAULT THRESHOLD, CHECK TRACKING FOR EACH VIDEO, PLOTTING DATA ###
    use_roi = 'y' #input('draw ROI? y/n ')
    check_tracking = 'n' # input('check eye tracking for each video? y/n ')
    plot_data = 'y' #input('plot each trial once analysed? y/n ')
    set_threshold = 'y'

    ### ROI SELECTION ###
    if use_roi == 'y':
        video = Video(os.path.join(folder, avis[0]))
        print(video)
        ROI = selectROI(video)
    else:
        ROI = None

    ### SETTING NEW THRESHOLD ###
    if set_threshold == 'y':
        video = Video(os.path.join(folder, avis[0]))
        thresh = setThreshold(video, 200, ROI)
    else:
        thresh = 200#threshold[count]

    ### ANALYSIS STARTS HERE (LOOP THROUGH EACH AVI FILE IN SELECTED FOLDER) ###
    for avi in avis:

        ### CREATE FILEPATH TO VIDEO FILE ###
        file_path = os.path.join(folder, avi)
        print("ANALYZING", avi)
        name = os.path.splitext(avi)[0]

        ### CREATE AN OUTPUT PATH TO SAVE RESULTS ###
        output_path = os.path.join(output_folder, name + '.csv')

        ### IMPROT VIDEO ###
        video = Video(file_path)
        video.name = avi
        ### CHECK TRACKING ###
        if check_tracking == 'y':
            checkTracking(video, thresh, ROI)

        ### MAIN ANALYSIS FUNCTION ###
        data = analyseVideo(video, thresh, ROI)

        ### PLOTTING ###
        if plot_data == 'y':
            data.plot()
            plt.savefig(output_folder + '/' + name + '.png')
            plt.close()
        ### SAVE RESULTS ###
        data.to_csv(output_path)

ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-0%_T12.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-0%_T5.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-0%_T6.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-20%_T15.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-20%_T4.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-20%_T7.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-40%_T11.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-40%_T3.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-40%_T9.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-60%_T14.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-60%_T2.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-60%_T8.avi
ANALYZING 2P094-A5_9deg@5ds0dsy_RL-5s-LR_30s_200deg_30dots_contrs-80%_T1.av